## Import des données et pré-traitement

In [ ]:
import pandas as pd
from src.utility.descriptive_statistics import descriptive_statistics

df = pd.read_pickle('src/data/panel_data.pkl')
df["index"] = pd.to_datetime(df["index"])

outlier_dates = [pd.Timestamp('2001-09-11')]
df = df[~df['index'].isin(outlier_dates)]

df = df[(df['index'] >= '1988-01-01') & (df['index'] <= '2017-01-01')]

for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

## Statistiques descriptives

In [ ]:
import numpy as np

original_stats, log_stats = descriptive_statistics(df["Maturity 1"])

stats_keys = ['Mean', 'Median', 'Minimum', 'Maximum', 'Std deviation', 'Skewness', 'Kurtosis', 'Autocorrelation', 'ADF test p-value (10 lags)', 'Nb obs']
df_combined_stats = pd.DataFrame(index=stats_keys, columns=['Prices (c/bu)', 'Log returns'])

for key in stats_keys:
    df_combined_stats.loc[key, 'Prices (c/bu)'] = original_stats.get(key, np.nan)
    log_key = 'Log ' + key  
    df_combined_stats.loc[key, 'Log returns'] = log_stats.get(log_key, np.nan)
df_combined_stats

## Graphiques


In [ ]:
import matplotlib.pyplot as plt
import os 

df.set_index('index', inplace=True)

target_date = '2004-06-17'

prix_17_06_2004 = df.loc[target_date, ['Maturity 1', 'Maturity 2', 'Maturity 3', 'Maturity 4', 'Maturity 5']]

plt.plot(['Maturity 1', 'Maturity 2', 'Maturity 3', 'Maturity 4', 'Maturity 5'], prix_17_06_2004 , marker='o', linestyle='-')
plt.title(f'Term Structure of Corn Futures Prices, {target_date}')
plt.grid()

output_dir = 'static/graph'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_path = os.path.join(output_dir, f'term_structure_{target_date}.png')
plt.savefig(output_path)
plt.show()


In [ ]:
from src.graph.graph import plot_and_save_graph

plot_and_save_graph(df, ["Maturity 1"], 
                    "Front-month settlement price (cents/bu)", 
                    "Dates", "cents/bu", 
                    output_filename='price_history.png', 
                    output_dir='static/graph')

# Estimation du modele espace d'etat


## Preparation des données

In [ ]:
import pandas as pd
import numpy as np
from src.utility.date import get_T, get_t
df.index = pd.to_datetime(df.index)
df[['T1', 'T2', 'T3', 'T4', 'T5']] = pd.DataFrame(df.index.map(lambda x: pd.Series(get_T(x))).tolist(), index=df.index)
for i in range(1, 6):
    df[f'Maturity {i}'] = np.log(df[f'Maturity {i}'])
df
df = df[(df.index >= '1988-02-01') & (df.index <= '2015-01-01')]

df['t'] = df.index.to_series().apply(get_t)
df


## Estimation des coefficients de la composante saisonière

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Convertir l'index en datetime
df.index = pd.to_datetime(df.index)

y = df['Maturity 1']

df['Cos1'] = np.cos(2 * np.pi * df.index.dayofyear / 365.25)
df['Sin1'] = np.sin(2 * np.pi * df.index.dayofyear / 365.25)
df['Cos2'] = np.cos(4 * np.pi * df.index.dayofyear / 365.25)
df['Sin2'] = np.sin(4 * np.pi * df.index.dayofyear / 365.25)

X = df[['Cos1', 'Sin1', 'Cos2', 'Sin2']]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

# Extraire les coefficients estimés
coefficients = model.params
seasonal_coeffs = {
    'coeff_Cos1': coefficients[1],
    'coeff_Sin1': coefficients[2],
    'coeff_Cos2': coefficients[3],
    'coeff_Sin2': coefficients[4]
}

## Estimation du modèle espace d'Etat

In [ ]:
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
from tqdm import tqdm
from src.model.performance import calculate_performance
from src.model.kalman import KalmanModel
from src.utility.parameter import calculate_num_parameters, mu, kappa, sigma1, sigma2, rho, lambdaz, x1

def objective(params, observations, times, maturities, n_factors, seasonal_coeffs):
    param_keys = ['x1_initial', 'mu', 'sigma1', 'lambda1', 'kappa2', 'sigma2', 'lambda2', 'rho12',
                  'kappa3', 'sigma3', 'lambda3', 'rho13', 'rho23',
                  'kappa4', 'sigma4', 'lambda4', 'rho14', 'rho24', 'rho34']
    num_params = calculate_num_parameters(n_factors) + 1  
    model_params = {
        key: params[i] for i, key in enumerate(param_keys[:num_params])
    }
    model_params['maturities'] = maturities
    model_params['current_time'] = times
    model_params['seasonal_coeffs'] = seasonal_coeffs

    for key in model_params:
        if 'kappa' in key and model_params[key] == 0 and n_factors > 1:
            raise ValueError(f"{key} cannot be zero for {n_factors}-factor model")

    model = KalmanModel(n_factors=n_factors, params=model_params, seasonal_coeffs=seasonal_coeffs)
    return model.compute_likelihood(observations, times, maturities)

# Paramètres et données d'observation
observations = df.iloc[:, 0:5].values
maturities = df.iloc[:, 6:11].values
times = df['t'].values

# Estimations initiales des paramètres
initial_guesses = {
    1: [x1, mu, sigma1, lambdaz],
    2: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho],
    3: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho, kappa, sigma2, lambdaz, rho, rho],
    4: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho, kappa, sigma2, lambdaz, rho, rho, kappa, sigma2, lambdaz, rho, rho, rho]
}

# Optimisation et calcul des performances
results = {}
rmse_results = {}
for n_factors in tqdm(range(1, 5)):
    initial_result = minimize(
        objective,
        initial_guesses[n_factors],
        args=(observations, times, maturities, n_factors, seasonal_coeffs),
        method='Nelder-Mead',
        options={'maxiter': 1}
    )

    final_result = minimize(
        objective,
        initial_result.x,
        args=(observations, times, maturities, n_factors, seasonal_coeffs),
        method='BFGS',
        options={'maxiter': 1}
    )
    
    results[n_factors] = final_result
    print(f"Optimized parameters for {n_factors} factors:", final_result.x)

    hessian_inv = final_result.hess_inv
    if isinstance(hessian_inv, np.ndarray):
        covariance_matrix = hessian_inv
    else:
        covariance_matrix = hessian_inv.todense()

    try:
        np.linalg.cholesky(covariance_matrix)
        std_errors = np.sqrt(np.diag(covariance_matrix))
    except np.linalg.LinAlgError:
        std_errors = np.full(covariance_matrix.shape[0], np.inf)

    z_values = final_result.x / std_errors
    p_values = [2 * (1 - stats.norm.cdf(np.abs(z))) for z in z_values]

    num_params = calculate_num_parameters(n_factors) + 1  
    param_keys = ['x1_initial', 'mu', 'sigma1', 'lambda1', 'kappa2', 'sigma2', 'lambda2', 'rho12',
                  'kappa3', 'sigma3', 'lambda3', 'rho13', 'rho23',
                  'kappa4', 'sigma4', 'lambda4', 'rho14', 'rho24', 'rho34']
    param_keys = param_keys[:num_params]

    for i, (param, std_err, p_value) in enumerate(zip(final_result.x, std_errors, p_values)):
        print(f"Parameter {param_keys[i]}: estimate={param}, std_error={std_err}, p_value={p_value}")

    rmse_results[n_factors] = calculate_performance(n_factors, final_result.x, param_keys, observations, times, maturities, seasonal_coeffs)

# Affichage des résultats RMSE
for n_factors, rmses in rmse_results.items():
    print(f"RMSE for {n_factors} factors: {rmses}")

## Estimation du modèle débruité


In [ ]:
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
from tqdm import tqdm
from src.model.performance import calculate_performance
from src.model.kalman import KalmanModel
from src.utility.parameter import calculate_num_parameters, mu, kappa, sigma1, sigma2, rho, lambdaz, x1
from src.model.wavelet import denoise_all_signal 

def objective(params, observations, times, maturities, n_factors, seasonal_coeffs):
    param_keys = ['x1_initial', 'mu', 'sigma1', 'lambda1', 'kappa2', 'sigma2', 'lambda2', 'rho12',
                  'kappa3', 'sigma3', 'lambda3', 'rho13', 'rho23',
                  'kappa4', 'sigma4', 'lambda4', 'rho14', 'rho24', 'rho34']
    num_params = calculate_num_parameters(n_factors) + 1  
    model_params = {
        key: params[i] for i, key in enumerate(param_keys[:num_params])
    }
    model_params['maturities'] = maturities
    model_params['current_time'] = times
    model_params['seasonal_coeffs'] = seasonal_coeffs

    for key in model_params:
        if 'kappa' in key and model_params[key] == 0 and n_factors > 1:
            raise ValueError(f"{key} cannot be zero for {n_factors}-factor model")

    model = KalmanModel(n_factors=n_factors, params=model_params, seasonal_coeffs=seasonal_coeffs)
    return model.compute_likelihood(observations, times, maturities)

df_denoised = denoise_all_signal(df, wavelet='db1', level=1)

observations = df_denoised.iloc[:, 0:5].values
maturities = df_denoised.iloc[:, 6:11].values
times = df_denoised['t'].values

# Estimations initiales des paramètres
initial_guesses = {
    1: [x1, mu, sigma1, lambdaz],
    2: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho],
    3: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho, kappa, sigma2, lambdaz, rho, rho],
    4: [x1, mu, sigma1, lambdaz, kappa, sigma2, lambdaz, rho, kappa, sigma2, lambdaz, rho, rho, kappa, sigma2, lambdaz, rho, rho, rho]
}

# Optimisation et calcul des performances
results = {}
rmse_results = {}
for n_factors in tqdm(range(1, 5)):
    initial_result = minimize(
        objective,
        initial_guesses[n_factors],
        args=(observations, times, maturities, n_factors, seasonal_coeffs),
        method='Nelder-Mead',
        options={'maxiter': 1}
    )

    final_result = minimize(
        objective,
        initial_result.x,
        args=(observations, times, maturities, n_factors, seasonal_coeffs),
        method='BFGS',
        options={'maxiter': 1}
    )
    
    results[n_factors] = final_result
    print(f"Optimized parameters for {n_factors} factors:", final_result.x)

    hessian_inv = final_result.hess_inv
    if isinstance(hessian_inv, np.ndarray):
        covariance_matrix = hessian_inv
    else:
        covariance_matrix = hessian_inv.todense()

    try:
        np.linalg.cholesky(covariance_matrix)
        std_errors = np.sqrt(np.diag(covariance_matrix))
    except np.linalg.LinAlgError:
        std_errors = np.full(covariance_matrix.shape[0], np.inf)

    z_values = final_result.x / std_errors
    p_values = [2 * (1 - stats.norm.cdf(np.abs(z))) for z in z_values]

    num_params = calculate_num_parameters(n_factors) + 1  
    param_keys = ['x1_initial', 'mu', 'sigma1', 'lambda1', 'kappa2', 'sigma2', 'lambda2', 'rho12',
                  'kappa3', 'sigma3', 'lambda3', 'rho13', 'rho23',
                  'kappa4', 'sigma4', 'lambda4', 'rho14', 'rho24', 'rho34']
    param_keys = param_keys[:num_params]

    for i, (param, std_err, p_value) in enumerate(zip(final_result.x, std_errors, p_values)):
        print(f"Parameter {param_keys[i]}: estimate={param}, std_error={std_err}, p_value={p_value}")

    rmse_results[n_factors] = calculate_performance(n_factors, final_result.x, param_keys, observations, times, maturities, seasonal_coeffs)

# Affichage des résultats RMSE
for n_factors, rmses in rmse_results.items():
    print(f"RMSE for {n_factors} factors: {rmses}")


## Out of sample forecast